In [39]:
import sys
import pandas as pd
import numpy as np
import json

import os
from os import path
import time
from dateutil.relativedelta import relativedelta
# from datetime import timedelta
from datetime import datetime, timezone, timedelta
import requests
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
%matplotlib inline

In [2]:
def transform_df(df):
   
    for col in df.columns:
        df[col] = df[col].apply(str)
    
    mydict = df.to_dict('index')

    return mydict


def thingsboard_http(mydict):
    address = "https://mi6.meazon.com"
    dev_token = 'aaDmhlZQakTN4fEnK7UA' # 
    
    r = requests.post(address + "/api/auth/login",
                      json={'username': 'meazonpro@meazon.com', 'password': 'meazonpro1'}).json()
    acc_token = 'Bearer' + ' ' + r['token']


    for key, value in mydict.items():
        my_json = json.dumps({'ts': key, 'values': value})
        r = requests.post(url=address + "/api/v1/" + dev_token + "/telemetry",
                           data=my_json, headers={'Content-Type': 'application/json', 'Accept': '*/*',
                                                  'X-Authorization': acc_token})
                                                  
    print('Finished writing descriptors')
            
    return

In [69]:
address = 'https://apps.deddie.gr/AppWS/AppServices/loginPub'
# data = {"usr":"Johngionas","pwd":"jG88-k11","flg":"pub","appname":"MeteringDataPortalLegalPerson"}

r = requests.post(address,data = {"usr":"Johngionas","pwd":"jG88-k11","flg":"pub","appname":"MeteringDataPortalLegalPerson"},
              headers={'Content-Type': 'application/x-www-form-urlencoded', "charset":"utf-8","X-Requested-With": "XMLHttpRequest", 'Accept': '*/*'}).json()

token = r['token']
token

'f7e4b9f363721db8a8f8834d4c0e608cafed6722'

In [75]:
start = ['2019-12-31T22:00:00.000Z',
         '2020-01-31T22:00:00.000Z',
         '2020-02-29T22:00:00.000Z',
         '2020-03-31T21:00:00.000Z',
         '2020-04-30T21:00:00.000Z',
         '2020-05-31T21:00:00.000Z',
         '2020-06-30T21:00:00.000Z',
         '2020-07-31T21:00:00.000Z',
         '2020-08-31T21:00:00.000Z',
         '2020-09-30T21:00:00.000Z',
         '2020-10-31T22:00:00.000Z',
         '2020-11-30T22:00:00.000Z',
         '2020-12-31T22:00:00.000Z',
         '2021-01-31T22:00:00.000Z',
         '2021-02-28T22:00:00.000Z',
         '2021-03-31T21:00:00.000Z',
         '2021-04-30T21:00:00.000Z',
         '2021-05-31T21:00:00.000Z',
         '2021-06-30T21:00:00.000Z',
         '2021-07-31T21:00:00.000Z',
         '2021-08-31T21:00:00.000Z',
         '2021-09-30T21:00:00.000Z',
         '2021-10-31T22:00:00.000Z',
         '2021-11-30T22:00:00.000Z',
         '2021-12-31T22:00:00.000Z'
        ]

In [84]:
summary = pd.DataFrame([])

for i in range(0,len(start)-1):

    address = 'https://apps.deddie.gr/mdp/rest/getCurves/'
    data = {"supplyNumber":"787758020","taxNumber":"999627480","fromDate":start[i],"toDate":start[i+1],"classType":"reactive","quarterAnalysisFlag":True,"confirmedDataFlag":True}
    
    r = requests.post(address,data = json.dumps(data),
                  headers={'Content-Type': 'application/json', 'Accept': '*/*', 'token': token}).json()
    
    df = pd.DataFrame(r['curves'])
    
    df['date'] = pd.to_datetime(df['meterDate'], format='%d/%m/%Y %H:%M')
    df['date'] = df['date']-timedelta(minutes=15)
    df['date'] = df['date'].dt.tz_localize('Europe/Athens',ambiguous=False,nonexistent='NaT')
    df = df.loc[df['date'].notnull()]
    df.set_index('date', drop=True, inplace=True)
    df.drop('meterDate',axis=1, inplace=True)
    
    summary = pd.concat([summary,df])
    print(summary.tail())
    

                           consumption
date                                  
2020-02-01 22:45:00+02:00         4.32
2020-02-01 23:00:00+02:00         4.56
2020-02-01 23:15:00+02:00         5.12
2020-02-01 23:30:00+02:00         4.80
2020-02-01 23:45:00+02:00         4.20
                           consumption
date                                  
2020-03-01 22:45:00+02:00         3.92
2020-03-01 23:00:00+02:00         4.00
2020-03-01 23:15:00+02:00         4.28
2020-03-01 23:30:00+02:00         5.44
2020-03-01 23:45:00+02:00         5.56
                           consumption
date                                  
2020-04-01 22:45:00+03:00         3.96
2020-04-01 23:00:00+03:00         3.92
2020-04-01 23:15:00+03:00         3.04
2020-04-01 23:30:00+03:00         3.08
2020-04-01 23:45:00+03:00         2.96
                           consumption
date                                  
2020-05-01 22:45:00+03:00         3.68
2020-05-01 23:00:00+03:00         3.24
2020-05-01 23:15:00+03:00

In [85]:
df['consumption'] = df['consumption'].cumsum()
df.rename(columns={'consumption':'rcnrgA'},inplace=True)
df.head()

,rcnrgA
date,
2021-12-01 00:00:00+02:00,4.60
2021-12-01 00:15:00+02:00,8.24
2021-12-01 00:30:00+02:00,11.16
2021-12-01 00:45:00+02:00,14.48
2021-12-01 01:00:00+02:00,17.52


In [86]:
df['ts'] = df.index.astype(np.int64) // 10 ** 6
df.set_index('ts', drop=True, inplace=True)
df.head()

,rcnrgA
ts,
1638309600000,4.60
1638310500000,8.24
1638311400000,11.16
1638312300000,14.48
1638313200000,17.52


In [87]:
mydict = transform_df(df)
thingsboard_http(mydict)

Finished writing descriptors
